# ESM-2 150M Fine-tuning (Enhanced Architecture)

**Purpose:** Corrected pipeline with:
- **Three separate heads** (MF, BP, CC) instead of single head
- **Attention pooling** instead of mean pooling
- **Layer freezing schedule** (freeze 90% for epoch 1, then unfreeze)
- **Enhanced AsymmetricLoss** with label smoothing
- **Per-aspect thresholds** (MF=0.40, BP=0.06, CC=0.30)

**Expected improvement:** Better BP performance through dedicated head + lower threshold.

## 1. Environment and Imports

In [ ]:
import os
from pathlib import Path
import json
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from transformers import AutoTokenizer, EsmModel, get_linear_schedule_with_warmup
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.2)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Device check
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Imports successful")
print(f"🖥️  Device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

In [ ]:
ENVIRONMENT = 'local'

# Handle running from notebooks/ or project root
if Path.cwd().name == 'notebooks':
    BASE_DIR = Path.cwd().parent
else:
    BASE_DIR = Path.cwd()

CONFIG = {
    # Model
    'model_name': 'facebook/esm2_t30_150M_UR50D',  # 150M parameters
    
    # Training
    'batch_size': 2,
    'gradient_accumulation': 16,  # Effective batch: 32
    'learning_rate': 1e-5,
    'weight_decay': 0.01,
    'num_epochs': 3,
    'warmup_ratio': 0.1,
    'max_length': 448,
    
    # Model architecture
    'dropout': 0.3,
    'freeze_ratio': 0.9,  # Freeze 90% of layers in epoch 1
    
    # Asymmetric Loss
    'gamma_neg': 4.0,
    'gamma_pos': 1.0,
    'clip': 0.05,
    'label_smoothing': 0.01,
    
    # Per-aspect thresholds (from enhanced config)
    'thresholds': {
        'MF': 0.40,
        'BP': 0.06,  # Lower for BP to capture more predictions
        'CC': 0.30
    },
    
    # Paths
    'data_dir': BASE_DIR,
    'output_dir': BASE_DIR / 'models' / 'esm_threehead',
    'train_fasta': BASE_DIR / 'Train' / 'train_sequences.fasta',
    'train_terms': BASE_DIR / 'Train' / 'train_terms.tsv',
    'obo_path': BASE_DIR / 'Train' / 'go-basic.obo',
    
    # Reproducibility
    'seed': 42,
    'val_split': 0.1,
    
    # Mixed precision
    'use_fp16': True
}

# Create output directory
CONFIG['output_dir'].mkdir(parents=True, exist_ok=True)

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## 3. Data Loaders (Ontology, Sequences, Labels)

In [ ]:
class OntologyLoader:
    """Load GO ontology and build aspect mappings."""
    
    def __init__(self, obo_path: str):
        self.obo_path = obo_path
        self.term_to_aspect = {}
        self.aspect_terms = {"MF": set(), "BP": set(), "CC": set()}
        self._load()
    
    def _load(self):
        current_id = None
        current_namespace = None
        namespace_map = {
            "molecular_function": "MF",
            "biological_process": "BP", 
            "cellular_component": "CC"
        }
        
        with open(self.obo_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith("[Term]"):
                    current_id = None
                    current_namespace = None
                elif line.startswith("id: GO:"):
                    current_id = line.split("id: ")[1]
                elif line.startswith("namespace:"):
                    ns = line.split("namespace: ")[1]
                    current_namespace = namespace_map.get(ns)
                    if current_id and current_namespace:
                        self.term_to_aspect[current_id] = current_namespace
                        self.aspect_terms[current_namespace].add(current_id)
        
        print(f"Loaded {len(self.term_to_aspect)} GO terms")
        for asp, terms in self.aspect_terms.items():
            print(f"  {asp}: {len(terms)} terms")


class SequenceLoader:
    """Load protein sequences from FASTA files."""
    
    @staticmethod
    def load_fasta(fasta_path: str) -> Dict[str, str]:
        sequences = {}
        current_id = None
        current_seq = []
        
        with open(fasta_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if current_id:
                        sequences[current_id] = ''.join(current_seq)
                    current_id = line[1:].split()[0]
                    current_seq = []
                else:
                    current_seq.append(line)
            
            if current_id:
                sequences[current_id] = ''.join(current_seq)
        
        print(f"Loaded {len(sequences)} sequences from {Path(fasta_path).name}")
        return sequences


class LabelLoader:
    """Load GO term annotations."""
    
    @staticmethod
    def load_terms(terms_path: str) -> Dict[str, List[str]]:
        protein_terms = {}
        
        with open(terms_path, 'r') as f:
            next(f)  # Skip header
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 2:
                    protein_id, term = parts[0], parts[1]
                    if protein_id not in protein_terms:
                        protein_terms[protein_id] = []
                    protein_terms[protein_id].append(term)
        
        print(f"Loaded annotations for {len(protein_terms)} proteins")
        return protein_terms

## 4. Load Data and Build Vocabulary

In [ ]:
# Load ontology
ontology = OntologyLoader(CONFIG['obo_path'])

# Load sequences
train_sequences = SequenceLoader.load_fasta(CONFIG['train_fasta'])

# Load labels
protein_terms = LabelLoader.load_terms(CONFIG['train_terms'])

# Build vocabulary from training data
all_terms = set()
for terms in protein_terms.values():
    all_terms.update(terms)

# Filter to terms in ontology and sort
valid_terms = sorted([t for t in all_terms if t in ontology.term_to_aspect])
term_to_idx = {t: i for i, t in enumerate(valid_terms)}
idx_to_term = {i: t for t, i in term_to_idx.items()}

# Build aspect indices
aspect_indices = {"MF": [], "BP": [], "CC": []}
for term, idx in term_to_idx.items():
    aspect = ontology.term_to_aspect[term]
    aspect_indices[aspect].append(idx)

print(f"\nVocabulary: {len(valid_terms)} terms")
for asp, indices in aspect_indices.items():
    print(f"  {asp}: {len(indices)} terms")

## 5. Dataset Class

In [ ]:
class ProteinDataset(Dataset):
    """Dataset for protein sequences with GO term labels."""
    
    def __init__(
        self,
        protein_ids: List[str],
        sequences: Dict[str, str],
        protein_terms: Dict[str, List[str]],
        term_to_idx: Dict[str, int],
        tokenizer,
        max_length: int = 512
    ):
        self.protein_ids = protein_ids
        self.sequences = sequences
        self.protein_terms = protein_terms
        self.term_to_idx = term_to_idx
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.num_labels = len(term_to_idx)
    
    def __len__(self):
        return len(self.protein_ids)
    
    def __getitem__(self, idx):
        protein_id = self.protein_ids[idx]
        sequence = self.sequences[protein_id]
        
        # Truncate sequence if needed
        if len(sequence) > self.max_length - 2:  # Account for special tokens
            sequence = sequence[:self.max_length - 2]
        
        # Tokenize
        encoding = self.tokenizer(
            sequence,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Build label vector
        labels = torch.zeros(self.num_labels, dtype=torch.float32)
        if protein_id in self.protein_terms:
            for term in self.protein_terms[protein_id]:
                if term in self.term_to_idx:
                    labels[self.term_to_idx[term]] = 1.0
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': labels,
            'protein_id': protein_id
        }

## 6. Model Architecture (Three-Head with Attention Pooling)

In [ ]:
class AttentionPool(nn.Module):
    """Attention-based pooling layer."""
    
    def __init__(self, hidden_size: int):
        super().__init__()
        self.attention = nn.Linear(hidden_size, 1)
    
    def forward(self, hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        # hidden_states: (batch, seq_len, hidden)
        # attention_mask: (batch, seq_len)
        
        # Compute attention scores
        scores = self.attention(hidden_states).squeeze(-1)  # (batch, seq_len)
        
        # Mask padding tokens
        scores = scores.masked_fill(attention_mask == 0, float('-inf'))
        
        # Softmax
        weights = F.softmax(scores, dim=-1).unsqueeze(-1)  # (batch, seq_len, 1)
        
        # Weighted sum
        pooled = (hidden_states * weights).sum(dim=1)  # (batch, hidden)
        
        return pooled


class ESMThreeHeadModel(nn.Module):
    """ESM-2 with three separate classification heads (MF, BP, CC)."""
    
    def __init__(
        self,
        model_name: str,
        aspect_indices: Dict[str, List[int]],
        dropout: float = 0.3
    ):
        super().__init__()
        
        # Load ESM-2 backbone
        self.esm = EsmModel.from_pretrained(model_name)
        hidden_size = self.esm.config.hidden_size
        
        # Store aspect info
        self.aspect_indices = aspect_indices
        self.num_mf = len(aspect_indices['MF'])
        self.num_bp = len(aspect_indices['BP'])
        self.num_cc = len(aspect_indices['CC'])
        
        # Attention pooling
        self.attention_pool = AttentionPool(hidden_size)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Three separate heads
        self.head_mf = nn.Linear(hidden_size, self.num_mf)
        self.head_bp = nn.Linear(hidden_size, self.num_bp)
        self.head_cc = nn.Linear(hidden_size, self.num_cc)
        
        print(f"Model initialized:")
        print(f"  Hidden size: {hidden_size}")
        print(f"  MF head: {self.num_mf} outputs")
        print(f"  BP head: {self.num_bp} outputs")
        print(f"  CC head: {self.num_cc} outputs")
    
    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor
    ) -> Dict[str, torch.Tensor]:
        # Get ESM hidden states
        outputs = self.esm(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # (batch, seq_len, hidden)
        
        # Attention pooling
        pooled = self.attention_pool(hidden_states, attention_mask)  # (batch, hidden)
        pooled = self.dropout(pooled)
        
        # Three heads
        logits_mf = self.head_mf(pooled)  # (batch, num_mf)
        logits_bp = self.head_bp(pooled)  # (batch, num_bp)
        logits_cc = self.head_cc(pooled)  # (batch, num_cc)
        
        return {
            'MF': logits_mf,
            'BP': logits_bp,
            'CC': logits_cc
        }

## 7. Asymmetric Loss Function

In [ ]:
class AsymmetricLossV2(nn.Module):
    """
    Canonical Asymmetric Loss for multi-label classification.
    
    Paper: "Asymmetric Loss For Multi-Label Classification" (Ridnik et al., 2021)
    
    Key idea: Down-weight easy negatives more aggressively than positives
    to handle extreme class imbalance in multi-label settings.
    """
    
    def __init__(
        self,
        gamma_neg: float = 4.0,
        gamma_pos: float = 1.0,
        clip: float = 0.05,
        eps: float = 1e-8,
        label_smoothing: float = 0.0
    ):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
        self.label_smoothing = label_smoothing
    
    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits: Raw model outputs (batch, num_classes)
            targets: Binary labels (batch, num_classes)
        
        Returns:
            Scalar loss
        """
        # Apply label smoothing
        if self.label_smoothing > 0:
            targets = targets * (1 - self.label_smoothing) + 0.5 * self.label_smoothing
        
        # Sigmoid
        probs = torch.sigmoid(logits)
        probs_pos = probs
        probs_neg = 1 - probs
        
        # Asymmetric clipping (shift negatives down)
        if self.clip > 0:
            probs_neg = (probs_neg + self.clip).clamp(max=1)
        
        # Focal weights
        # For positives: (1 - p)^gamma_pos
        # For negatives: p^gamma_neg (after clipping)
        weight_pos = (1 - probs_pos) ** self.gamma_pos
        weight_neg = probs_pos ** self.gamma_neg
        
        # Cross entropy components
        loss_pos = -targets * weight_pos * torch.log(probs_pos + self.eps)
        loss_neg = -(1 - targets) * weight_neg * torch.log(probs_neg + self.eps)
        
        loss = loss_pos + loss_neg
        
        return loss.mean()

## 8. Data Preparation

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])

# Filter to proteins with both sequence and annotations
valid_proteins = [
    pid for pid in train_sequences.keys()
    if pid in protein_terms and len(protein_terms[pid]) > 0
]
print(f"Valid proteins: {len(valid_proteins)}")

# Train/val split
random.seed(CONFIG['seed'])
random.shuffle(valid_proteins)
split_idx = int(len(valid_proteins) * (1 - CONFIG['val_split']))
train_ids = valid_proteins[:split_idx]
val_ids = valid_proteins[split_idx:]

print(f"Train: {len(train_ids)}, Val: {len(val_ids)}")

# Create datasets
train_dataset = ProteinDataset(
    protein_ids=train_ids,
    sequences=train_sequences,
    protein_terms=protein_terms,
    term_to_idx=term_to_idx,
    tokenizer=tokenizer,
    max_length=CONFIG['max_length']
)

val_dataset = ProteinDataset(
    protein_ids=val_ids,
    sequences=train_sequences,
    protein_terms=protein_terms,
    term_to_idx=term_to_idx,
    tokenizer=tokenizer,
    max_length=CONFIG['max_length']
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print(f"\nTrain batches: {len(train_loader)}, Val batches: {len(val_loader)}")

## 9. Model, Optimizer, and Training Setup

In [ ]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Initialize model
model = ESMThreeHeadModel(
    model_name=CONFIG['model_name'],
    aspect_indices=aspect_indices,
    dropout=CONFIG['dropout']
)
model = model.to(device)

# Loss function
criterion = AsymmetricLossV2(
    gamma_neg=CONFIG['gamma_neg'],
    gamma_pos=CONFIG['gamma_pos'],
    clip=CONFIG['clip'],
    label_smoothing=CONFIG['label_smoothing']
)

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Scheduler
total_steps = len(train_loader) * CONFIG['num_epochs'] // CONFIG['gradient_accumulation']
warmup_steps = int(total_steps * CONFIG['warmup_ratio'])

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Mixed precision scaler
scaler = GradScaler() if CONFIG['use_fp16'] else None

print(f"\nTraining setup:")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Gradient accumulation: {CONFIG['gradient_accumulation']}")
print(f"  Effective batch size: {CONFIG['batch_size'] * CONFIG['gradient_accumulation']}")

## 10. Layer Freezing Utilities

In [ ]:
def freeze_esm_layers(model: ESMThreeHeadModel, freeze_ratio: float = 0.9):
    """
    Freeze a proportion of ESM layers from the bottom.
    
    Args:
        model: ESMThreeHeadModel instance
        freeze_ratio: Proportion of layers to freeze (0.0 = none, 1.0 = all)
    """
    # Get encoder layers
    encoder_layers = model.esm.encoder.layer
    num_layers = len(encoder_layers)
    num_freeze = int(num_layers * freeze_ratio)
    
    # Freeze embeddings
    for param in model.esm.embeddings.parameters():
        param.requires_grad = False
    
    # Freeze bottom layers
    for i in range(num_freeze):
        for param in encoder_layers[i].parameters():
            param.requires_grad = False
    
    # Unfreeze top layers
    for i in range(num_freeze, num_layers):
        for param in encoder_layers[i].parameters():
            param.requires_grad = True
    
    # Classification heads always trainable
    for param in model.attention_pool.parameters():
        param.requires_grad = True
    for param in model.head_mf.parameters():
        param.requires_grad = True
    for param in model.head_bp.parameters():
        param.requires_grad = True
    for param in model.head_cc.parameters():
        param.requires_grad = True
    
    # Count parameters
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    
    print(f"Layer freezing: {num_freeze}/{num_layers} layers frozen")
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")


def unfreeze_all(model: ESMThreeHeadModel):
    """Unfreeze all model parameters."""
    for param in model.parameters():
        param.requires_grad = True
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"All layers unfrozen. Trainable params: {trainable:,}")

## 11. Evaluation Function

In [ ]:
def evaluate_model(
    model: ESMThreeHeadModel,
    dataloader: DataLoader,
    device: torch.device,
    aspect_indices: Dict[str, List[int]],
    thresholds: Dict[str, float]
) -> Dict[str, float]:
    """
    Evaluate model with aspect-specific thresholds.
    
    Returns dict with per-aspect and overall F1 scores.
    """
    model.eval()
    
    all_preds = {asp: [] for asp in ['MF', 'BP', 'CC']}
    all_labels = {asp: [] for asp in ['MF', 'BP', 'CC']}
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            with autocast(enabled=CONFIG['use_fp16']):
                logits = model(input_ids, attention_mask)
            
            # Extract predictions for each aspect
            for asp in ['MF', 'BP', 'CC']:
                asp_indices = aspect_indices[asp]
                asp_labels = labels[:, asp_indices].cpu().numpy()
                asp_probs = torch.sigmoid(logits[asp]).cpu().numpy()
                
                all_labels[asp].append(asp_labels)
                all_preds[asp].append(asp_probs)
    
    # Concatenate
    for asp in ['MF', 'BP', 'CC']:
        all_labels[asp] = np.vstack(all_labels[asp])
        all_preds[asp] = np.vstack(all_preds[asp])
    
    # Calculate F1 per aspect
    results = {}
    f1_scores = []
    
    for asp in ['MF', 'BP', 'CC']:
        thresh = thresholds[asp]
        binary_preds = (all_preds[asp] >= thresh).astype(int)
        
        # Sample-wise F1
        f1_samples = []
        for i in range(len(binary_preds)):
            pred_set = set(np.where(binary_preds[i] == 1)[0])
            true_set = set(np.where(all_labels[asp][i] == 1)[0])
            
            if len(pred_set) == 0 and len(true_set) == 0:
                f1_samples.append(1.0)
            elif len(pred_set) == 0 or len(true_set) == 0:
                f1_samples.append(0.0)
            else:
                precision = len(pred_set & true_set) / len(pred_set)
                recall = len(pred_set & true_set) / len(true_set)
                if precision + recall > 0:
                    f1_samples.append(2 * precision * recall / (precision + recall))
                else:
                    f1_samples.append(0.0)
        
        results[f'{asp}_f1'] = np.mean(f1_samples)
        f1_scores.append(results[f'{asp}_f1'])
    
    # Overall F1 (weighted by aspect)
    results['overall_f1'] = np.mean(f1_scores)
    
    return results

## 12. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'MF_f1': [],
    'BP_f1': [],
    'CC_f1': [],
    'overall_f1': []
}

best_f1 = 0.0
best_epoch = 0

# Evaluation thresholds
eval_thresholds = CONFIG['thresholds'].copy()

print("=" * 60)
print("TRAINING START")
print("=" * 60)

for epoch in range(CONFIG['num_epochs']):
    print(f"\nEpoch {epoch + 1}/{CONFIG['num_epochs']}")
    print("-" * 40)
    
    # Layer freezing schedule
    if epoch == 0:
        freeze_esm_layers(model, freeze_ratio=CONFIG['freeze_ratio'])
    elif epoch == 1 and CONFIG['freeze_ratio'] > 0:
        unfreeze_all(model)
        # Reduce learning rate when unfreezing
        for param_group in optimizer.param_groups:
            param_group['lr'] = CONFIG['learning_rate'] * 0.5
        print(f"LR reduced to {CONFIG['learning_rate'] * 0.5}")
    
    # Training phase
    model.train()
    train_loss = 0.0
    optimizer.zero_grad()
    
    pbar = tqdm(train_loader, desc="Training")
    for step, batch in enumerate(pbar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass with mixed precision
        with autocast(enabled=CONFIG['use_fp16']):
            logits = model(input_ids, attention_mask)
            
            # Compute loss for each head
            loss_mf = criterion(logits['MF'], labels[:, aspect_indices['MF']])
            loss_bp = criterion(logits['BP'], labels[:, aspect_indices['BP']])
            loss_cc = criterion(logits['CC'], labels[:, aspect_indices['CC']])
            
            loss = (loss_mf + loss_bp + loss_cc) / 3
            loss = loss / CONFIG['gradient_accumulation']
        
        # Backward pass
        if CONFIG['use_fp16']:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        train_loss += loss.item() * CONFIG['gradient_accumulation']
        
        # Gradient accumulation step
        if (step + 1) % CONFIG['gradient_accumulation'] == 0:
            if CONFIG['use_fp16']:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            scheduler.step()
            optimizer.zero_grad()
        
        pbar.set_postfix({'loss': f'{train_loss / (step + 1):.4f}'})
    
    avg_train_loss = train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            with autocast(enabled=CONFIG['use_fp16']):
                logits = model(input_ids, attention_mask)
                
                loss_mf = criterion(logits['MF'], labels[:, aspect_indices['MF']])
                loss_bp = criterion(logits['BP'], labels[:, aspect_indices['BP']])
                loss_cc = criterion(logits['CC'], labels[:, aspect_indices['CC']])
                
                loss = (loss_mf + loss_bp + loss_cc) / 3
            
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)
    
    # Evaluate F1 scores
    eval_results = evaluate_model(model, val_loader, device, aspect_indices, eval_thresholds)
    
    history['MF_f1'].append(eval_results['MF_f1'])
    history['BP_f1'].append(eval_results['BP_f1'])
    history['CC_f1'].append(eval_results['CC_f1'])
    history['overall_f1'].append(eval_results['overall_f1'])
    
    print(f"\nEpoch {epoch + 1} Results:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")
    print(f"  MF F1:      {eval_results['MF_f1']:.4f}")
    print(f"  BP F1:      {eval_results['BP_f1']:.4f}")
    print(f"  CC F1:      {eval_results['CC_f1']:.4f}")
    print(f"  Overall F1: {eval_results['overall_f1']:.4f}")
    
    # Save best model
    if eval_results['overall_f1'] > best_f1:
        best_f1 = eval_results['overall_f1']
        best_epoch = epoch + 1
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_f1': best_f1,
            'config': CONFIG,
            'term_to_idx': term_to_idx,
            'aspect_indices': aspect_indices
        }, CONFIG['output_dir'] / 'best_model.pt')
        
        print(f"  *** New best model saved! ***")

print("\n" + "=" * 60)
print(f"TRAINING COMPLETE")
print(f"Best F1: {best_f1:.4f} at epoch {best_epoch}")
print("=" * 60)

## 13. Training Visualisation

## 12b. Compare Threshold Configurations

Evaluate both threshold sets to determine which performs better:
- **Enhanced thresholds**: MF=0.40, BP=0.06, CC=0.30 (aggressive BP)
- **KNN-optimal thresholds**: MF=0.40, BP=0.20, CC=0.40 (conservative)

In [ ]:
# Define threshold configurations to compare
THRESHOLD_CONFIGS = {
    'enhanced': {'MF': 0.40, 'BP': 0.06, 'CC': 0.30},   # Aggressive BP
    'knn_optimal': {'MF': 0.40, 'BP': 0.20, 'CC': 0.40}  # Conservative (KNN-tuned)
}

# Reload best model
checkpoint = torch.load(CONFIG['output_dir'] / 'best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print("Best model reloaded for threshold comparison\n")

# Evaluate each threshold configuration
threshold_results = {}

for config_name, thresholds in THRESHOLD_CONFIGS.items():
    print(f"Evaluating with '{config_name}' thresholds: {thresholds}")
    results = evaluate_model(model, val_loader, device, aspect_indices, thresholds)
    threshold_results[config_name] = results
    print(f"  MF: {results['MF_f1']:.4f}, BP: {results['BP_f1']:.4f}, CC: {results['CC_f1']:.4f}")
    print(f"  Overall F1: {results['overall_f1']:.4f}\n")

# Comparison table
print("=" * 60)
print("THRESHOLD COMPARISON SUMMARY")
print("=" * 60)
print(f"{'Config':<15} {'MF':<10} {'BP':<10} {'CC':<10} {'Overall':<10}")
print("-" * 55)
for config_name, results in threshold_results.items():
    print(f"{config_name:<15} {results['MF_f1']:.4f}     {results['BP_f1']:.4f}     {results['CC_f1']:.4f}     {results['overall_f1']:.4f}")
print("-" * 55)

# Determine best
best_config = max(threshold_results.items(), key=lambda x: x[1]['overall_f1'])
print(f"\n✅ Best config: '{best_config[0]}' with F1 = {best_config[1]['overall_f1']:.4f}")

# Store best thresholds for test prediction
BEST_THRESHOLDS = THRESHOLD_CONFIGS[best_config[0]]
print(f"   Using thresholds: {BEST_THRESHOLDS}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1 = axes[0]
epochs_range = range(1, len(history['train_loss']) + 1)
ax1.plot(epochs_range, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
ax1.plot(epochs_range, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# F1 plot
ax2 = axes[1]
ax2.plot(epochs_range, history['MF_f1'], 'g-', label='MF F1', linewidth=2, marker='o')
ax2.plot(epochs_range, history['BP_f1'], 'b-', label='BP F1', linewidth=2, marker='s')
ax2.plot(epochs_range, history['CC_f1'], 'purple', label='CC F1', linewidth=2, marker='^')
ax2.plot(epochs_range, history['overall_f1'], 'r-', label='Overall F1', linewidth=2.5, marker='D')
ax2.axhline(y=0.2579, color='gray', linestyle='--', label='KNN Baseline', alpha=0.7)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('F1 Score', fontsize=12)
ax2.set_title('Validation F1 Scores by Aspect', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(CONFIG['output_dir'] / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nPlot saved to {CONFIG['output_dir'] / 'training_curves.png'}")

## 14. Generate Test Predictions

In [ ]:
GENERATE_TEST_PREDICTIONS = True

if GENERATE_TEST_PREDICTIONS:
    print("Loading best model for test predictions...")
    
    # Load best model
    checkpoint = torch.load(CONFIG['output_dir'] / 'best_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    # Load test sequences
    test_fasta = Path(CONFIG['data_dir']) / 'Test' / 'testsuperset.fasta'
    test_sequences = SequenceLoader.load_fasta(test_fasta)
    test_ids = list(test_sequences.keys())
    
    print(f"Test proteins: {len(test_ids)}")
    
    # Create test dataset (no labels needed)
    class TestDataset(Dataset):
        def __init__(self, protein_ids, sequences, tokenizer, max_length):
            self.protein_ids = protein_ids
            self.sequences = sequences
            self.tokenizer = tokenizer
            self.max_length = max_length
        
        def __len__(self):
            return len(self.protein_ids)
        
        def __getitem__(self, idx):
            protein_id = self.protein_ids[idx]
            sequence = self.sequences[protein_id]
            
            if len(sequence) > self.max_length - 2:
                sequence = sequence[:self.max_length - 2]
            
            encoding = self.tokenizer(
                sequence,
                padding='max_length',
                truncation=True,
                max_length=self.max_length,
                return_tensors='pt'
            )
            
            return {
                'input_ids': encoding['input_ids'].squeeze(0),
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'protein_id': protein_id
            }
    
    test_dataset = TestDataset(test_ids, test_sequences, tokenizer, CONFIG['max_length'])
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0)
    
    # Generate predictions
    all_predictions = []
    
    print("Generating predictions...")
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test inference"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            protein_ids = batch['protein_id']
            
            with autocast(enabled=CONFIG['use_fp16']):
                logits = model(input_ids, attention_mask)
            
            # Convert to probabilities
            probs_mf = torch.sigmoid(logits['MF']).cpu().numpy()
            probs_bp = torch.sigmoid(logits['BP']).cpu().numpy()
            probs_cc = torch.sigmoid(logits['CC']).cpu().numpy()
            
            # Create full probability vector
            for i, pid in enumerate(protein_ids):
                full_probs = np.zeros(len(term_to_idx))
                full_probs[aspect_indices['MF']] = probs_mf[i]
                full_probs[aspect_indices['BP']] = probs_bp[i]
                full_probs[aspect_indices['CC']] = probs_cc[i]
                
                # Store predictions above threshold for each term
                # Use BEST_THRESHOLDS from comparison (falls back to CONFIG if not defined)
                active_thresholds = BEST_THRESHOLDS if 'BEST_THRESHOLDS' in dir() else CONFIG['thresholds']
                for term, idx in term_to_idx.items():
                    prob = full_probs[idx]
                    asp = ontology.term_to_aspect[term]
                    thresh = active_thresholds[asp]
                    
                    if prob >= thresh:
                        all_predictions.append({
                            'protein_id': pid,
                            'term': term,
                            'probability': float(prob),
                            'aspect': asp
                        })
    
    # Save predictions
    test_pred_df = pd.DataFrame(all_predictions)
    test_pred_path = CONFIG['output_dir'] / 'test_predictions.parquet'
    test_pred_df.to_parquet(test_pred_path, index=False)
    
    print(f"\nTest predictions saved: {test_pred_path}")
    print(f"Total predictions: {len(test_pred_df):,}")
    print(f"Unique proteins: {test_pred_df['protein_id'].nunique():,}")
    print(f"\nPredictions by aspect:")
    print(test_pred_df.groupby('aspect').size())
else:
    print("Test prediction generation skipped. Set GENERATE_TEST_PREDICTIONS = True to enable.")

## 15. Save Metadata and Summary

In [ ]:
# Save training history
history_df = pd.DataFrame(history)
history_df.to_csv(CONFIG['output_dir'] / 'training_history.csv', index=False)

# Save metadata for submission pipeline
# Use best thresholds from comparison if available
active_thresholds = BEST_THRESHOLDS if 'BEST_THRESHOLDS' in dir() else CONFIG['thresholds']

metadata = {
    'model_name': 'esm_threehead_150M',
    'model_path': str(CONFIG['output_dir'] / 'best_model.pt'),
    'aspect_specific_f1': {
        'MF': history['MF_f1'][best_epoch - 1],
        'BP': history['BP_f1'][best_epoch - 1],
        'CC': history['CC_f1'][best_epoch - 1]
    },
    'overall_f1': best_f1,
    'best_epoch': best_epoch,
    'optimal_thresholds': active_thresholds,
    'threshold_comparison': threshold_results if 'threshold_results' in dir() else None,
    'config': {k: str(v) if isinstance(v, Path) else v for k, v in CONFIG.items()},
    'num_terms': len(term_to_idx),
    'architecture': {
        'backbone': CONFIG['model_name'],
        'pooling': 'attention',
        'heads': 3,
        'loss': 'AsymmetricLossV2',
        'freeze_ratio': CONFIG['freeze_ratio']
    }
}

import json
with open(CONFIG['output_dir'] / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)
print(f"Model:          ESM-2 150M Three-Head")
print(f"Best Epoch:     {best_epoch}")
print(f"Best F1:        {best_f1:.4f}")
print(f"  MF F1:        {metadata['aspect_specific_f1']['MF']:.4f}")
print(f"  BP F1:        {metadata['aspect_specific_f1']['BP']:.4f}")
print(f"  CC F1:        {metadata['aspect_specific_f1']['CC']:.4f}")
print(f"KNN Baseline:   0.2579")
print(f"Improvement:    {(best_f1 - 0.2579) / 0.2579 * 100:+.1f}%")
print("=" * 60)
print(f"\nOutputs saved to: {CONFIG['output_dir']}")
print(f"  - best_model.pt")
print(f"  - training_history.csv")
print(f"  - training_curves.png")
print(f"  - metadata.json")
if GENERATE_TEST_PREDICTIONS:
    print(f"  - test_predictions.parquet")